<a href="https://colab.research.google.com/github/dhruvRajoria/python-projects/blob/main/AdaptiveRouteNavigator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import math
import time
import threading
import random
from queue import PriorityQueue
from typing import List, Tuple, Dict

# Traffic levels affecting speed (km/h)
class TrafficLevel:
    LOW = 80
    MEDIUM = 50
    HIGH = 20

# Location structure
class Location:
    def __init__(self, name: str, latitude: float, longitude: float):
        self.name = name
        self.latitude = latitude
        self.longitude = longitude

# Edge structure for traffic-aware graph
class Edge:
    def __init__(self, to: str, base_distance: float, traffic: int):
        self.to = to
        self.base_distance = base_distance
        self.traffic = traffic

    # Calculate travel time based on traffic
    def travel_time(self) -> float:
        return self.base_distance / self.traffic  # Time = Distance / Speed

# Graph class
class TrafficGraph:
    def __init__(self):
        self.adj_list: Dict[str, List[Edge]] = {}
        self.locations: List[Location] = []

    @staticmethod
    def to_radians(degree: float) -> float:
        return degree * math.pi / 180.0

    def haversine(self, loc1: Location, loc2: Location) -> float:
        lat1, lon1 = self.to_radians(loc1.latitude), self.to_radians(loc1.longitude)
        lat2, lon2 = self.to_radians(loc2.latitude), self.to_radians(loc2.longitude)

        dlat = lat2 - lat1
        dlon = lon2 - lon1

        a = (math.sin(dlat / 2) ** 2 +
             math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2)

        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
        R = 6371.0  # Radius of Earth in kilometers
        return R * c

    def heuristic(self, current: str, goal: str) -> float:
        loc1 = self.get_location(current)
        loc2 = self.get_location(goal)
        return self.haversine(loc1, loc2) if loc1 and loc2 else float('inf')

    def get_location(self, name: str) -> Location:
        for loc in self.locations:
            if loc.name == name:
                return loc
        return None

    def simulate_traffic(self):
        for edges in self.adj_list.values():
            for edge in edges:
                edge.traffic = random.choice([TrafficLevel.LOW, TrafficLevel.MEDIUM, TrafficLevel.HIGH])

    def add_location(self, loc: Location):
        self.locations.append(loc)
        self.adj_list[loc.name] = []

    def add_edge(self, from_loc: str, to_loc: str):
        loc1 = self.get_location(from_loc)
        loc2 = self.get_location(to_loc)
        if loc1 and loc2:
            distance = self.haversine(loc1, loc2)
            self.adj_list[from_loc].append(Edge(to_loc, distance, TrafficLevel.MEDIUM))
            self.adj_list[to_loc].append(Edge(from_loc, distance, TrafficLevel.MEDIUM))

    def dijkstra(self, start: str, end: str) -> Tuple[List[str], float, float]:
        distance = {loc: float('inf') for loc in self.adj_list}
        time = {loc: float('inf') for loc in self.adj_list}
        parent = {}
        pq = PriorityQueue()

        distance[start] = 0.0
        time[start] = 0.0
        pq.put((0.0, start))

        while not pq.empty():
            _, current = pq.get()

            if current == end:
                break

            for edge in self.adj_list[current]:
                new_dist = distance[current] + edge.base_distance
                new_time = time[current] + edge.travel_time()

                if new_dist < distance[edge.to]:
                    distance[edge.to] = new_dist
                    time[edge.to] = new_time
                    parent[edge.to] = current
                    pq.put((new_time, edge.to))

        path = []
        cost = distance[end]
        eta = time[end]
        if cost != float('inf'):
            at = end
            while at != start:
                path.append(at)
                at = parent.get(at, None)
            path.append(start)
            path.reverse()

        return path, cost, eta

    def find_and_print_path(self, start: str, end: str):
        path, cost, eta = self.dijkstra(start, end)

        if cost == float('inf'):
            print(f"No path found between {start} and {end}")
        else:
            print(f"Path: {' -> '.join(path)}")
            print(f"Total Distance: {cost:.2f} km")
            print(f"Estimated Time of Arrival: {eta:.2f} hours")

    def print_graph(self):
        print("Graph:")
        for from_loc, edges in self.adj_list.items():
            print(f"{from_loc}: ", end="")
            for edge in edges:
                print(f"{edge.to} (Distance: {edge.base_distance:.2f} km, Speed: {edge.traffic} km/h), ", end="")
            print()

    def start_traffic_simulation(self):
        def simulate():
            while True:
                time.sleep(10)
                self.simulate_traffic()
                print("Traffic updated!")

        threading.Thread(target=simulate, daemon=True).start()

# Main function
def main():
    graph = TrafficGraph()

    graph.add_location(Location("A", 12.971598, 77.594566))  # Bangalore
    graph.add_location(Location("B", 13.082680, 80.270718))  # Chennai
    graph.add_location(Location("C", 17.385044, 78.486671))  # Hyderabad
    graph.add_location(Location("D", 19.076090, 72.877426))  # Mumbai
    graph.add_location(Location("E", 28.704060, 77.102493))  # Delhi

    graph.add_edge("A", "B")
    graph.add_edge("A", "C")
    graph.add_edge("B", "D")
    graph.add_edge("C", "D")
    graph.add_edge("D", "E")

    graph.start_traffic_simulation()
    graph.print_graph()

    start = input("Enter the starting location: ")
    end = input("Enter the ending location: ")

    graph.find_and_print_path(start, end)

if __name__ == "__main__":
    main()


Graph:
A: B (Distance: 290.18 km, Speed: 50 km/h), C (Distance: 500.00 km, Speed: 50 km/h), 
B: A (Distance: 290.18 km, Speed: 50 km/h), D (Distance: 1033.13 km, Speed: 50 km/h), 
C: A (Distance: 500.00 km, Speed: 50 km/h), D (Distance: 621.49 km, Speed: 50 km/h), 
D: B (Distance: 1033.13 km, Speed: 50 km/h), C (Distance: 621.49 km, Speed: 50 km/h), E (Distance: 1153.24 km, Speed: 50 km/h), 
E: D (Distance: 1153.24 km, Speed: 50 km/h), 
Traffic updated!
Enter the starting location: A
Traffic updated!
Enter the ending location: D
Path: A -> C -> D
Total Distance: 1121.49 km
Estimated Time of Arrival: 22.43 hours
